In [4]:
import os
import re
import numpy as np
import tifffile as tiff
import cv2

In [5]:
def max_projection_montage(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)  # Ensure output folder exists
    
    # Regex to match files containing '-F4' to '-F11' and extract Name1_name2 part
    pattern = re.compile(r'(.+)_([^-]+) - (.+)_(\w+_\w+)\.tif')
    
    grouped_files = {}
    
    # Group files by Name1_name2
    for filename in os.listdir(input_folder):
        match = pattern.match(filename)
        if filename.endswith(".tif") and match:
            base_name, time, specific, name_pair = match.groups()
            if name_pair not in grouped_files:
                grouped_files[name_pair] = []
            grouped_files[name_pair].append((time, filename))
    
    for name_pair, file_list in grouped_files.items():
        file_list.sort()  # Sort by time for ordered montage
        projections = []

        output_path = os.path.join(output_folder, f"{name_pair}_montage.tif")
        # Skip processing if the montage file already exists
        if os.path.exists(output_path):
            print(f"Skipping {name_pair}, already processed.")
            continue
        
        for time, filename in file_list:
            input_path = os.path.join(input_folder, filename)
            
            # Read multi-dimensional TIF file with metadata
            with tiff.TiffFile(input_path) as tif_file:
                img = tif_file.asarray()
                metadata = tif_file.pages[0].tags
                resolution_tag = metadata.get('XResolution')  # Get resolution if available
                pixel_size = 0.1883734  # Default pixel size in microns
                if resolution_tag:
                    resolution = resolution_tag.value  # Extract actual value
                    pixel_size = resolution[1] / resolution[0]  # Convert to microns per pixel
            
            if len(img.shape) == 4 and img.shape[1] == 4:  # (Z, C, H, W) format
                img = np.moveaxis(img, 1, 0)  # Convert to (C, Z, H, W)
                
                # Max project channel 3 (index 2) and channel 2 (index 1) only using z 4-16
                img = img[:, 5:16, :, :]               
                ch3 = np.max(img[2], axis=0)
                ch2 = np.max(img[1], axis=0)
                
                # Find the brightest object in channel 3
                min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(ch3)
                center_x, center_y = max_loc
                
                if max_val == 0:
                    continue
                
                # Define cropping bounds (200x200 pixels around the brightest object)
                half_size = 100
                h, w = ch3.shape
                x1, x2 = center_x - half_size, center_x + half_size
                y1, y2 = center_y - half_size, center_y + half_size
                
                pad_x1, pad_x2, pad_y1, pad_y2 = 0, 0, 0, 0
                if x1 < 0:
                    pad_x1 = abs(x1)
                    x1 = 0
                if x2 > w:
                    pad_x2 = x2 - w
                    x2 = w
                if y1 < 0:
                    pad_y1 = abs(y1)
                    y1 = 0
                if y2 > h:
                    pad_y2 = y2 - h
                    y2 = h
                
                ch3_crop = ch3[y1:y2, x1:x2]
                ch2_crop = ch2[y1:y2, x1:x2]
                
                ch3_padded = cv2.copyMakeBorder(ch3_crop, pad_y1, pad_y2, pad_x1, pad_x2, cv2.BORDER_CONSTANT, value=0)
                ch2_padded = cv2.copyMakeBorder(ch2_crop, pad_y1, pad_y2, pad_x1, pad_x2, cv2.BORDER_CONSTANT, value=0)
                
                # Normalize and merge channels
                def safe_normalize(img):
                    if np.max(img) == np.min(img):
                        return np.zeros_like(img, dtype=np.uint8)
                    return cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                
                ch3_gray = safe_normalize(ch3_padded)
                ch2_gray = safe_normalize(ch2_padded)
               # merged = cv2.merge((ch3_gray, ch2_gray, np.zeros_like(ch3_gray)))  # Merge as RGB

                # Create the merged image with ch3 in cyan and ch2 in grayscale
                merged = cv2.merge((ch3_gray, ch3_gray, np.zeros_like(ch3_gray)))  # Cyan channel
                grayscale = cv2.merge((ch2_gray, ch2_gray, ch2_gray))  # Grayscale channel
                    
                # Combine the cyan and grayscale images using max blending to preserve both
                final_merged = np.maximum(merged, grayscale)
                
                projections.append((final_merged, filename, time))
        
        if projections:
            # Sort projections by time and type
            im_projections = [p for p in projections if "_IM_" in p[1]]
            cm_projections = [p for p in projections if "_CM_" in p[1]]
            
            # Arrange in 16-column by 3-row grid
            sorted_projections = {"24h": [], "48h": [], "72h": []}
            for proj in im_projections + cm_projections:
                for key in sorted_projections:
                    if key in proj[1]:
                        sorted_projections[key].append(proj)
                        break
            
            cols = 16
            rows = 3
            img_h, img_w, _ = projections[0][0].shape
            montage = np.zeros((rows * img_h, cols * img_w, 3), dtype=np.uint8)
            
            for row_idx, key in enumerate(["24h", "48h", "72h"]):
                row_projections = sorted_projections[key]
                for idx, (img, filename, _) in enumerate(row_projections):
                    if "_IM_" in filename:
                        col = idx % 8  # Keep within the first 8 columns
                    else:
                        col = (idx % 8) + 8  # Keep within the last 8 columns
                    y1, y2 = row_idx * img_h, (row_idx + 1) * img_h
                    x1, x2 = col * img_w, (col + 1) * img_w
                    
                    montage[y1:y2, x1:x2] = img
                  #  filename_short = filename.split("ReScan_")[1]
                    
                  #  cv2.putText(montage, filename_short, (x1 + 5, y1 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1) # labeling obsolete when the grid order is correct
            
            
            with tiff.TiffWriter(output_path) as tif:
                tif.write(montage, metadata={'axes': 'YX', 'XResolution': (1, pixel_size), 'YResolution': (1, pixel_size)})
            
            print(f'Created montage: {output_path}')

In [6]:
input_folder = "/Volumes/arxivBeta/_Tobias/Opera/20250204/01_Importer"  
output_folder = "/Volumes/arxivBeta/_Tobias/Opera/20250204/03_Galleries_V5" 

max_projection_montage(input_folder, output_folder)

Skipping CALM1_init, already processed.
Skipping CAMSAP1_init, already processed.
Skipping KIF11_orig, already processed.
Skipping ANAPC11_new, already processed.
Skipping APC_new, already processed.
Skipping ASPM_init, already processed.
Skipping CENPJ_init, already processed.
Skipping CENPI_orig, already processed.
Skipping CENPF_orig, already processed.
Skipping CENPE_orig, already processed.
Skipping CENPC_orig, already processed.
Skipping AURKA_orig, already processed.
Skipping AURKB_orig, already processed.
Skipping AURKC_init, already processed.
Skipping BIRC5_orig, already processed.
Skipping BUB1_orig, already processed.
Skipping CDK5RAP2_init, already processed.
Skipping CDK1_orig, already processed.
Skipping CDCA8_orig, already processed.
Skipping CDC42_orig, already processed.
Skipping CDC34_init, already processed.
Skipping BUB1B_orig, already processed.
Skipping BUB3_init, already processed.
Skipping CDC20_orig, already processed.
Skipping CDC16_orig, already processed.
S